# Notebook to prepare the Eigenstrat of D. Reich
Idea: Keep it as Code Sequence, so I can readily update in case data gets updated

### Overview of structure of this notebook

1. Download the Data
2. Test 

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from matplotlib import gridspec
import socket
import os as os
import sys as sys
import multiprocessing as mp

socket_name = socket.gethostname()
print(socket_name)
if socket_name == "VioletQueen":
    path = "/home/harald/git/HAPSBURG/"   # The Path on Harald's machine
elif socket_name.startswith("midway2"):
    print("Midway jnovmbre partition detected.")
    path = "/project/jnovembre/hringbauer/HAPSBURG/"  # The Path on Midway Cluster
else: 
    raise RuntimeWarning("Not compatible machine. Check!!")
    
os.chdir(path)  # Set the right Path (in line with Atom default)

sys.path.append("./Python3/")  # Since now we are in the Root Directory
from hmm_inference import HMM_Analyze   # Do not move. Should be after sys.path..
#sys.path.append("./Python3/create1000G_Mosaic/")  # Since now we are in the Root Directory
#from createMosaicsMulti import Mosaic_1000G_Multi  # Import the object that can create the Multiruns

print(os.getcwd()) # Show the current working directory. Should be HAPSBURG/Notebooks/ParallelRuns
print(f"CPU Count: {mp.cpu_count()}")

midway2-0401.rcc.local
Midway jnovmbre partition detected.
/project/jnovembre/hringbauer/HAPSBURG
CPU Count: 28


In [7]:
### Download the Data
download_link = "https://reichdata.hms.harvard.edu/pub/datasets/amh_repo/curated_releases/V37/V37.2/SHARE/public.dir/v37.2.1240K.tar"
file_name = os.path.basename(download_link)  # Get the File Name
local_folder ="./Data/ReichLabEigenstrat/Raw/"
download_path = os.path.join(local_folder, file_name) 

if not os.path.exists(local_folder):
    print(f"Creating {local_folder}...")
    os.makedirs(local_folder)

### Run the Shell Download
!wget $download_link $download_path

--2019-09-04 14:50:20--  https://reichdata.hms.harvard.edu/pub/datasets/amh_repo/curated_releases/V37/V37.2/SHARE/public.dir/v37.2.1240K.tar
Resolving reichdata.hms.harvard.edu (reichdata.hms.harvard.edu)... 134.174.150.16
Connecting to reichdata.hms.harvard.edu (reichdata.hms.harvard.edu)|134.174.150.16|:443... failed: Connection timed out.
Retrying.

--2019-09-04 14:52:28--  (try: 2)  https://reichdata.hms.harvard.edu/pub/datasets/amh_repo/curated_releases/V37/V37.2/SHARE/public.dir/v37.2.1240K.tar
Connecting to reichdata.hms.harvard.edu (reichdata.hms.harvard.edu)|134.174.150.16|:443... ^C


In [8]:
print("Done")

Done


# Area 51
Test code here

In [4]:
download_link = "https://reichdata.hms.harvard.edu/pub/datasets/amh_repo/curated_releases/V37/V37.2/SHARE/public.dir/v37.2.1240K.tar"
os.path.basename(download_link)

'v37.2.1240K.tar'